In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2


In [ ]:
import genesis as gs
# show the iamge
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from genesis_gym import GenesisGym

In [ ]:
import random
import pathlib as pl
import numpy as np
import os, sys
import torch

## Default Args
DEFAULT_RADIUS = 0.034 # 0032
DEFAULT_HEIGHT = 0.09 # 0.1
DEFAULT_RHO = 2000
DEFAULT_FRICTION = 0.5
DEFAULT_STARTING_X = 0.65
    # parser = argparse.ArgumentParser(description='Genesis Gym Environment')
    # parser.add_argument('--vis', action='store_true', help='Enable visualization')
    # parser.add_argument('--radius', type=float, default=DEFAULT_RADIUS, help='Bottle radius')
    # parser.add_argument('-e', '--height', type=float, default=DEFAULT_HEIGHT, help='Bottle height')
    # parser.add_argument('-o', '--rho', type=float, default=DEFAULT_RHO, help='Density of the bottle')
    # parser.add_argument('--friction', type=float, default=DEFAULT_FRICTION, help='Friction of the bottle')
    # parser.add_argument('--starting_x', type=float, default=DEFAULT_STARTING_X, help='Starting x position of the bottle')
    # parser.add_argument('--max-demos', type=int, default=1e7, help='Max number of demos to load')
    # parser.add_argument('--random-agent', action='store_true', help='Use a random agent')
    # parser.add_argument('--subsample', type=int, default=2, help='Subsample ratio for the demos')
    # parser.add_argument('--env-name', type=str, default='lift', help='Environment name')
    # args = parser.parse_args()

args = {
    'vis': True,
    'radius': DEFAULT_RADIUS,
    'height': DEFAULT_HEIGHT,
    'rho': DEFAULT_RHO,
    'friction': DEFAULT_FRICTION,
    'starting_x': DEFAULT_STARTING_X,
    'max_demos': 1e7,
    'random_agent': False,
    'subsample': 2,
    'env_name': 'lift',
}

env = GenesisGym(**args)
obs = env.reset()

In [ ]:
from demo_holder import GenesisDemoHolder
from collections import defaultdict
demo_player = GenesisDemoHolder(use_eef=False, subsample_ratio=args['subsample'])
TRIAL_CAN_ADJUSTED = defaultdict(lambda: False)


In [ ]:
demo_player.next_demo()


In [ ]:
from genesis_gym import PZ
import time
demo_player.reset_current_demo()
env.scene.clear_debug_objects()


def get_grip_pose(env):
    # get the average position of the fingertips
    left_fingertip = env.kinova.get_link('left_finger_prox_link').get_pos().cpu().numpy()
    right_fingertip = env.kinova.get_link('right_finger_prox_link').get_pos().cpu().numpy()
    env.scene.draw_debug_sphere(left_fingertip, 0.01, color=(0, 1, 0))
    env.scene.draw_debug_sphere(right_fingertip, 0.01, color=(0, 1, 0))
    average_fingertip = np.mean([left_fingertip, right_fingertip], axis=0)

    offset_magnitude = 0.04
    # Get the end effector orientation in the world frame
    q = env.kinova.get_link('end_effector_link').get_quat().cpu().numpy()
    # Convert quaternion to rotation matrix
    """Converts quaternion to 3x3 rotation matrix."""
    d = q.dot(q)
    w, x, y, z = q
    s = 2 / d
    xs, ys, zs = x * s, y * s, z * s
    wx, wy, wz = w * xs, w * ys, w * zs
    xx, xy, xz = x * xs, x * ys, x * zs
    yy, yz, zz = y * ys, y * zs, z * zs

    R = np.array([[1 - (yy + zz), xy - wz, xz + wy], [xy + wz, 1 - (xx + zz), yz - wx], [xz - wy, yz + wx, 1 - (xx + yy)]])

    # Forward direction is the x-axis in the end effector frame
    forward_dir = R @ np.array([1, 0, 0])
    # Scale by offset magnitude and add to average fingertip position
    ret = average_fingertip + forward_dir * offset_magnitude
    env.scene.draw_debug_sphere(ret, 0.01, color=(1, 0, 0))
    env.scene.draw_debug_arrow(average_fingertip, ret - average_fingertip, radius=0.01, color=(1, 0, 0, 0.5))
    return ret
    # return np.mean([left_fingertip, right_fingertip], axis=0) + [0.04, 0., 0.]

trial_id = demo_player.get_trial_id()
TRIAL_CAN_ADJUSTED[trial_id] = False
env.reset(trial_id=trial_id)
action = demo_player.next_action()['action']
while action is not None:
    obs, reward, done, info = env.step(action)
    if args['vis']:
        env.render()

    # get the gripper position
    # print(f"action: {action[-1]:+1.2f} {obs['state'][-1]:+1.2f}", end=' ')


    # if the gripper action is closing and the can is nearby, move the can and restart the demo
    gripper_pos = env.kinova.get_link('end_effector_link').get_pos().cpu().numpy()
    can_pose = env.bottle.get_pos().cpu().numpy()
    dp = np.linalg.norm(gripper_pos - can_pose)
    if action[-1] > 50 and dp < 0.2 and not TRIAL_CAN_ADJUSTED[trial_id] and gripper_pos[2] < 0.12:
        print("Gripper closing and can is nearby, restarting demo and setting can to gripper pose")
        print(f"{action[-1]:+1.2f} {dp:+1.2f} {gripper_pos[2]:+1.2f} < 0.12")
        # get the average pos of the last 4 links 
        grip_pos = get_grip_pose(env)
        grip_pos[-1] = PZ
        # make a debug sphere
        debug_arrow = env.scene.draw_debug_arrow(pos=gripper_pos, vec=grip_pos - gripper_pos, radius=0.01, color=(1, 0, 0, 0.5))  # Green
        env.scene.draw_debug_sphere(gripper_pos, 0.01, color=(0, 1, 1))
        env.scene.draw_debug_sphere(grip_pos, 0.01, color=(0, 0, 1))
        env.reset(trial_id=trial_id)
        demo_player.reset_current_demo()

        for _ in range(10):
            env.scene.step() # let the arm get back before we reset the can

        env.set_can_to_pose(torch.Tensor(grip_pos))
        TRIAL_CAN_ADJUSTED[trial_id] = True

        time.sleep(10)

        env.scene.step()

    action = demo_player.next_action()
    action = action['action'] if action is not None else None
    if done:
        break




In [ ]:
demo_player.reset_current_demo()
env.reset(trial_id=trial_id)
env.step(demo_player.next_action()['action'])


In [ ]:
links = list(env.kinova.links)
for i, link in enumerate(links):
    print(f"{i}: {link.name}")

positions = []
for i, link in enumerate(links):
    if 'finger' in link.name and 'prox' not in link.name:
        positions.append(link.get_pos().cpu().numpy())


# get the average position of the fingertips
print(np.mean(positions, axis=0))

env.scene.draw_debug_sphere(np.mean(positions, axis=0), 0.01, color=(0, 1, 0))